In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # your 30x4 array
#y_init = np.array([...])  # your 30 outputs
X_init = datain
y_init = dataout
# --- Transform to maximization ---
y_trans = -y_init
y_best = y_trans.max()

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_trans)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we minimize in scipy
# --- Expected Improvement acquisition function ---

# --- Optimize acquisition function ---
bounds = [(0,1), (0,1), (0,1), (0,1)]
best_x = None
best_ei = float('inf')

# Multiple random starts to avoid local maxima
for _ in range(20):
    x0 = np.random.rand(4)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
import numpy as np

X_init = datain
y_init = dataout
# --- Transform to maximization ---
y_trans = -y_init
y_best = y_trans.max()


# Add the new observation
X_new = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])
y_new = np.array([-43.88421604431158])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

print(X_init)
print(y_init)


In [ ]:
# --- Transform to maximization (all values consistent) ---
y_trans_init = -y_init
y_new_trans = -y_new  # transform new point too

# --- Combine old + new data ---
X = np.vstack([X_init, x_new.reshape(1, -1)])
y = np.hstack([y_trans_init, y_new_trans])

# --- Current best ---
y_best = y.max()

print(X)
print(y)
print(y_best)


In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y)



In [ ]:

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we minimize in scipy

In [ ]:

# --- Bounds ---
bounds = [(0, 1)] * 4

# --- Optimize acquisition function ---
best_x = None
best_ei = float('inf')

for _ in range(15):
    # Random start in full space for exploration
    x0 = np.random.rand(4)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

for _ in range(5):
    # Random start near previous best for exploitation
    x0 = x_new + np.random.normal(0, 0.05, size=4)
    x0 = np.clip(x0, 0, 1)  # ensure within bounds
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)
